In [12]:
import gzip

import pandas as pd

pd.set_option("display.max_columns", None)

Read LOH positions

In [13]:
df = pd.read_csv("/home/junkhann/daten/LOH_positions_aggregated.tsv.gz", sep="\t", compression="gzip", low_memory=False)
# df = df.drop(columns=[""])
len(df)

235331

In [14]:
df.head()

,#CHROM,pos,REF,ALT,INFO,sample_control,sample_tumor,start,end,genotype,TCN,PID,normal_genotype,tumor_genotype,quality_score,reads_normal,reads_tumor,CELL_TYPE,RSID,GENE,GENE_ID,A1,A2,A2_FREQ_ONEK1K,A2_FREQ_HRC,position_code
0,1,842013,T,G,BRF=0.19;FR=0.25;HP=4;HapScore=1;MGOF=21;MMLQ=...,"1/0:-36.89,0,-58.19:21:99:33:13","0/0:0,-5.07,-54.5:0:51:18:0",840009,12839977,2:0,2.0,4134434,"1,1","2,0",99.000000,33,18,"Naïve/Immature B Cell,Memory B Cell,CD4 Effect...",rs7419119,{'UBE2J2'},{'ENSG00000160087'},T,G,0.21688,0.212812,1-842013
1,1,846398,G,A,BRF=0.44;FR=0.25;HP=3;HapScore=1;MGOF=29;MMLQ=...,"1/0:-23.69,0,-17.99:29:99:15:8","0/0:0,-5.12,-66.2:21:51:18:0",840009,12839977,2:0,2.0,4183136,"1,1","2,0",60.703590,15,18,"Naïve/Immature B Cell,Memory B Cell,CD4 Effect...",rs58781670,{'UBE2J2'},{'ENSG00000160087'},G,A,0.20774,0.216970,1-846398
2,1,848445,G,A,BRF=0.49;FR=0.254;HP=2;HapScore=3;MGOF=3;MMLQ=...,"1/0:-20.1,0,-34.3:3:99:15:5","0/0:0,-1.56,-51.1:2:16:16:0",840009,12839977,2:0,2.0,4183136,"1,1","2,0",53.958747,15,16,"Naïve/Immature B Cell,Memory B Cell,CD4 Effect...",rs4626817,{'UBE2J2'},{'ENSG00000160087'},G,A,0.20797,0.210548,1-848445
3,1,850780,C,T,BRF=0.17;FR=0.25;HP=1;HapScore=1;MGOF=4;MMLQ=2...,"0/1:-55.58,0,-49.58:4:99:46:23","0/0:0,-6.32,-76.5:1:63:25:0",840009,12839977,2:0,2.0,4149246,"1,1","2,0",99.000000,46,25,"Naïve/Immature B Cell,Memory B Cell,CD4 Effect...",rs6657440,{'UBE2J2'},{'ENSG00000160087'},C,T,0.60181,0.602125,1-850780
4,1,851190,G,A,BRF=0.29;FR=0.2501;HP=1;HapScore=1;MGOF=4;MMLQ...,"1/0:-42.59,0,-62.89:2:99:30:12","0/0:0,-3.12,-73.7:4:31:20:0",840009,12839977,2:0,2.0,4176325,"1,1","2,0",86.359655,30,20,"Naïve/Immature B Cell,Memory B Cell,CD4 Effect...",rs28609852,{'UBE2J2'},{'ENSG00000160087'},G,A,0.21290,0.213474,1-851190


Analyse LOH positions

In [15]:
no_distinct_positions = df["position_code"].nunique()
no_distinct_positions

223063

In [16]:
df["#CHROM"].nunique()

22

In [17]:
no_distinct_patients = df["PID"].nunique()
no_distinct_patients

204

In [18]:
unique_combinations = len(df.drop_duplicates(['PID','position_code']).index)
unique_combinations

235331

In [19]:
df["tumor_genotype"].value_counts()

tumor_genotype
1,0    107457
2,0    104513
4,0     15777
3,0      5687
5,0       746
0,1       606
6,0       502
0,2        37
0,3         4
8,0         1
0,4         1
Name: count, dtype: int64

Merge position count

In [21]:
ser = pd.read_json("/home/junkhann/daten/LOH_position_count_mmml.json", typ="series", dtype={str})
df_position_count = ser.to_frame("position_count")
df_position_count = df_position_count.reset_index(names="pos")
df_position_count.shape

(223063, 2)

In [22]:
df_position_count.head()

,pos,position_count
0,1-842013,1
1,1-846398,1
2,1-848445,1
3,1-850780,1
4,1-851190,1


In [23]:
df = df.merge(df_position_count, how="inner", left_on="position_code", right_on="pos")
len(df)

235331

Set threshold

In [28]:
threshold = 40
df_thresholded = df[df["position_count"] >= threshold]
len(df_thresholded)

209

Analyze thresholded dataframe

In [29]:
df_thresholded["PID"].nunique()

74

In [30]:
df_thresholded["#CHROM"].nunique()

6

In [31]:
df_thresholded["#CHROM"].unique()

array([ 1,  6,  9, 11, 17, 19])

In [32]:
df_thresholded["position_code"].nunique()

49

Write to csv

In [33]:
df_thresholded.to_csv(f"/home/junkhann/daten/LOH_pos_thresholded/LOH_positions_mmml_threshold_{threshold}.tsv.gz", sep="\t", header=True, index=False, compression="gzip")